## Imports

In [21]:
using CSV
using DataFrames
using MLJ
using GLM
using Lasso
using MLBase

### import base

In [20]:
# import base
using Pkg
Pkg.add("MLBase")

   Resolving package versions...
    Updating `~/.julia/environments/v1.7/Project.toml`
  [f0e99cf1] + MLBase v0.8.0
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`


## Load data

In [2]:
df = DataFrame(CSV.File("/Users/hilsts/Documents/linear-models-benchmark/data/song_data.csv"))

,song_name,song_popularity,song_duration_ms,acousticness,danceability
,String,Int64,Int64,Float64,Float64
1,Boulevard of Broken Dreams,73,262333,0.00552,0.496
2,In The End,66,216933,0.0103,0.542
3,Seven Nation Army,76,231733,0.00817,0.737
4,By The Way,74,216933,0.0264,0.451
5,How You Remind Me,56,223826,0.000954,0.447
6,Bring Me To Life,80,235893,0.00895,0.316
7,Last Resort,81,199893,0.000504,0.581
8,Are You Gonna Be My Girl,76,213800,0.00148,0.613
9,Mr. Brightside,80,222586,0.00108,0.33


In [5]:
df = select!(df, Not(:song_name))

,song_popularity,song_duration_ms,acousticness,danceability,energy,instrumentalness
,Int64,Int64,Float64,Float64,Float64,Float64
1,73,262333,0.00552,0.496,0.682,2.94e-5
2,66,216933,0.0103,0.542,0.853,0.0
3,76,231733,0.00817,0.737,0.463,0.447
4,74,216933,0.0264,0.451,0.97,0.00355
5,56,223826,0.000954,0.447,0.766,0.0
6,80,235893,0.00895,0.316,0.945,1.85e-6
7,81,199893,0.000504,0.581,0.887,0.00111
8,76,213800,0.00148,0.613,0.953,0.000582
9,80,222586,0.00108,0.33,0.936,0.0


## Linear Regression

In [6]:
using Lathe.preprocess: TrainTestSplit
train, test = TrainTestSplit(df,.75)

(14185×14 typename(DataFrame). Omitted printing of 10 columns
│ Row   │ song_popularity │ song_duration_ms │ acousticness │ danceability │
│       │ Int64           │ Int64            │ Float64      │ Float64      │
├───────┼─────────────────┼──────────────────┼──────────────┼──────────────┤
│ 1     │ 66              │ 216933           │ 0.0103       │ 0.542        │
│ 2     │ 74              │ 216933           │ 0.0264       │ 0.451        │
│ 3     │ 56              │ 223826           │ 0.000954     │ 0.447        │
│ 4     │ 80              │ 235893           │ 0.00895      │ 0.316        │
│ 5     │ 81              │ 199893           │ 0.000504     │ 0.581        │
│ 6     │ 76              │ 213800           │ 0.00148      │ 0.613        │
│ 7     │ 81              │ 203346           │ 0.00172      │ 0.542        │
│ 8     │ 78              │ 168253           │ 0.0424       │ 0.629        │
│ 9     │ 63              │ 185586           │ 0.0046       │ 0.496        │
│ 10    │ 75  

In [13]:
linearRegressor = lm(Term(:song_popularity) ~ sum(Term.(Symbol.(names(df[:, Not(:song_popularity)])))), df)
predict(linearRegressor, test)

4650-element Vector{Union{Missing, Float64}}:
 55.41466600194881
 55.28767145574357
 53.30550088420243
 52.1549596695484
 54.31439429104653
 48.43216344392008
 52.30381375653886
 51.23521803686206
 47.09652962239261
 52.28219885064443
 48.751269357310896
 51.44957424377385
 59.91712585443227
  ⋮
 55.27719834140903
 52.60293770941264
 48.320507282603806
 50.25498010816773
 51.36347002551752
 56.197753017889845
 53.70163916549002
 53.38079348974171
 48.811205563848745
 50.745196015683014
 54.399871132431066
 51.82471034158896

## Lasso

In [38]:
lasso = fit(LassoModel, Term(:song_popularity) ~ sum(Term.(Symbol.(names(train[:, Not(:song_popularity)])))), train)
predict(lasso, test)

4650-element Vector{Union{Missing, Float64}}:
 55.311705118488625
 55.39339321081738
 53.7078884490374
 51.81155407832282
 54.28213809987159
 49.435186448693685
 52.330442698888604
 51.66780202548399
 47.29156865928989
 52.06002307710682
 48.38100919510774
 51.477445252884785
 59.9125977984922
  ⋮
 54.86827337245871
 52.06044554310103
 48.46416368911684
 50.40371210467135
 51.11820692542112
 56.0346514237472
 53.260315886974055
 52.89409919132049
 48.50790467163573
 50.776563592831586
 54.53720363572094
 51.56871290400152

In [52]:
X_train = convert(Matrix, train[:, Not(:song_popularity)])
y_train = train.song_popularity
X_test = convert(Matrix, test[:, Not(:song_popularity)])
en = fit(LassoPath, X_train, y_train)
predict(en, X_test)

4650×49 Matrix{Float64}:
 52.9899  53.08    53.1621  53.2369  …  55.2706  55.2856  55.2991  55.3117
 52.9899  52.5604  52.1691  51.8125     55.3139  55.3429  55.3692  55.3934
 52.9899  53.0801  53.1622  53.237      53.7     53.7029  53.7056  53.7079
 52.9899  53.08    53.1622  53.237      51.8329  51.8251  51.8181  51.8116
 52.9899  53.0057  53.02    53.0331     54.2801  54.2808  54.2816  54.2821
 52.9899  53.08    53.162   53.2368  …  49.4536  49.4469  49.4407  49.4352
 52.9899  53.0801  53.1622  53.237      52.3571  52.3474  52.3386  52.3304
 52.9899  53.0778  53.1579  53.2309     51.6427  51.6518  51.6601  51.6678
 52.9899  52.9651  52.9425  52.9219     47.3931  47.3561  47.3224  47.2916
 52.9899  53.0798  53.1616  53.2362     52.0837  52.075   52.0672  52.06
 52.9899  52.8813  52.7823  52.6921  …  48.4395  48.4182  48.3988  48.381
 52.9899  53.0801  53.1622  53.237      51.4863  51.4831  51.4802  51.4774
 52.9899  53.0801  53.1622  53.237      59.8568  59.8772  59.8957  59.9126
  ⋮

In [44]:
X_train = convert(Matrix, train[:, Not(:song_popularity)])

14185×13 Matrix{Float64}:
 216933.0  0.0103    0.542  0.853  …  0.0  0.0498  105.256  4.0  0.37
 216933.0  0.0264    0.451  0.97      1.0  0.107   122.444  4.0  0.198
 223826.0  0.000954  0.447  0.766     1.0  0.0313  172.011  4.0  0.574
 235893.0  0.00895   0.316  0.945     0.0  0.124   189.931  4.0  0.32
 199893.0  0.000504  0.581  0.887     0.0  0.0624   90.578  4.0  0.724
 213800.0  0.00148   0.613  0.953  …  1.0  0.0855  105.046  4.0  0.537
 203346.0  0.00172   0.542  0.905     1.0  0.054   153.398  4.0  0.374
 168253.0  0.0424    0.629  0.897     1.0  0.0483  161.944  4.0  0.93
 185586.0  0.0046    0.496  0.863     1.0  0.0381  110.018  4.0  0.243
 209266.0  0.00434   0.647  0.964     0.0  0.06    126.942  4.0  0.875
 269000.0  0.0179    0.618  0.938  …  1.0  0.0456   91.455  4.0  0.875
 210240.0  0.000353  0.42   0.929     0.0  0.121   127.204  4.0  0.3
 237026.0  0.000423  0.278  0.676     1.0  0.0371  104.545  4.0  0.494
      ⋮                            ⋱                 ⋮  